<a href="https://colab.research.google.com/github/Nachuz33/eccd_assignments/blob/main/Copia_de_Cleaning_a_dataset_student.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Objective

The objective of this notebook is get hands-on experience on cleaning a "dirty" dataset.
Often, datasets are created from "free-text" fields. In free-text fields, data validation is not enforced and as a result, many conventions co-exist within the same column of data.
Dirty data can also ocurr when collection information from different sources. If these sources use different conventions to represent such data, additional efforts are required to homogenize it at a later stage.

## Setup

In [1]:
!pip install -q eccd_datasets pygradus

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.9/223.9 kB 14.7 MB/s eta 0:00:00


In [2]:
STUDENT_NAME = "Ignacio-Acuña"
COURSE_NAME = "eccd-oct23"
EXERCISE_NAME = "cleaning-a-dataset"

In [3]:
import pandas as pd
import numpy as np
from eccd_datasets import load_lingerie
from pygradus import create_exercise, check_solution

In [4]:
datasets = load_lingerie()

In [5]:
datasets.keys()

dict_keys(['ae_com', 'amazon_com', 'btemptd_com', 'calvinklein_com', 'hankypanky_com', 'macys_com', 'shop_nordstrom_com', 'us_topshop_com', 'victoriassecret_com'])

## The different datasets on their own

In [ ]:
datasets["ae_com"].head()

In [ ]:
datasets["amazon_com"].head()

### Joining all the datasets into one

In [8]:
df = pd.concat(datasets.values())

In [9]:
df.shape

(613143, 14)

# Calculating statistics on the dataset

## Unifying Victoria's Secret

In [19]:
def unify_victoria_secret(df):
    """
    We want that all brands that are related to Victoria's Secret
    have `victoria's secret` as their brand instead of what they
    currently have.
    """
    df = df.copy()
    new_string = "victoria's secret"

    df['brand_name'] = df['brand_name'].str.replace(r'.*victoria.*', new_string, case=False, regex=True)

    return df


In [20]:
print(df.columns)

Index(['product_name', 'mrp', 'price', 'pdp_url', 'brand_name',
       'product_category', 'retailer', 'description', 'rating', 'review_count',
       'style_attributes', 'total_sizes', 'available_size', 'color'],
      dtype='object')


In [21]:
df_unified = unify_victoria_secret(df)

print(df_unified.columns)

Index(['product_name', 'mrp', 'price', 'pdp_url', 'brand_name',
       'product_category', 'retailer', 'description', 'rating', 'review_count',
       'style_attributes', 'total_sizes', 'available_size', 'color'],
      dtype='object')


In [22]:
answer_victoria_secret =df_unified[df_unified["brand_name"] == "victoria's secret"].shape[0]
print(answer_victoria_secret)

453453


## Cleaning up the price

In this sectino we are going to transform the `price` column into a float column in USD dolars.

For this, be careful of the different formats in the data.

For simplicity, you might assume that all the prices are in USD dolars, regarding of the symbol of the currency used.

In [33]:
def clean_price(df):
    """
    In this function we will transform the
    `price` column into a column of floats.
    In case a product has more than one price,
    return the lowest one.
    """

    # Convert 'price' column to string
    df['price'] = df['price'].astype(str)

    # Remove non-numeric characters from 'price' column
    df['price'] = df['price'].str.replace(r'[^\d.]', '', regex=True)

    # Convert 'price' column to float
    df['price'] = pd.to_numeric(df['price'], errors='coerce')

    df = df.sort_values('price', ascending=True).drop_duplicates('product_name', keep='first')


    return df


In [34]:
df_clean = clean_price(df_unified)

In [35]:
print(df_clean.columns)

Index(['product_name', 'mrp', 'price', 'pdp_url', 'brand_name',
       'product_category', 'retailer', 'description', 'rating', 'review_count',
       'style_attributes', 'total_sizes', 'available_size', 'color'],
      dtype='object')


In [37]:
answer_unified_price_sum = df_clean["price"].sum()
print(answer_unified_price_sum)

93380.30000000002


In [ ]:
def low_high_product_mean(df):
    """
    Finally, we will calculate `product_category` with the lowest and highest mean price
    for the brand Victoria's Secret.
    """

    # Filter for Victoria's Secret brand
    df_vs = df[df["brand_name"] == "victoria's secret"]

    mean_prices = df_vs.groupby('product_category')['min_price'].agg(['mean', 'min'])

    lowest_mean = mean_prices.loc[mean_prices['mean'].idxmin(), 'min']

    # Find product category with highest mean price
    highest_mean = mean_prices.loc[mean_prices['mean'].idxmax(), 'idxmax']

    return lowest_mean, highest_mean


In [ ]:
lowest_mean, highest_mean = low_high_product_mean(df_clean)
assert np.allclose(lowest_mean, 3.6203030303030)
print("highest_mean", highest_mean)

KeyError: ignored

In [ ]:

proposed_solution = {
'attempt': {
    'course_name': COURSE_NAME,
    'exercise_name': EXERCISE_NAME,
    'username': STUDENT_NAME,
},
'task_attempts': [
         {
            "name": "victoria secret",
            "answer": str(answer_victoria_secret),
         },
         {
            "name": "price unification",
            "answer": str(answer_unified_price_sum),
         },
         {
            "name": "highest mean",
            "answer": str(highest_mean),
         },
]

}
check_solution(proposed_solution)


NameError: ignored